# Migration between administrative areas and overseas by sex

Tab: `In-Council Area-Sex`

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.nrscotland.gov.uk/statistics-and-data/statistics/' \
                      'statistics-by-theme/migration/migration-statistics/migration-between-scotland-and-overseas')
#     scraper.run()
    distribution = scraper.distribution(
        mediaType='application/vnd.ms-excel',
        title='Migration between administrative areas and overseas by sex')
    display(distribution)
    tab = [tab for tab in distribution.as_databaker() if tab.name == 'In-Council Area-Sex'][0]

In [2]:
observations = tab.excel_ref("B6").expand(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
observations 

{<C70 101.0>, <P49 168.0>, <P55 179.0>, <Q36 135.0>, <G82 182.0>, <M48 40.0>, <P29 938.0>, <B102 13.0>, <E68 278.0>, <I85 87.0>, <C93 795.0>, <F6 4459.0>, <M32 60.0>, <I111 437.0>, <J104 406.0>, <C32 33.0>, <C43 1349.0>, <J44 743.0>, <I20 10022.0>, <J52 72.0>, <K96 47.0>, <H86 259.0>, <P99 39.0>, <L100 66.0>, <B28 31.0>, <L88 58.0>, <Q21 730.0>, <Q11 173.0>, <Q32 53.0>, <K25 60.0>, <G85 135.0>, <N20 8897.0>, <M53 116.0>, <K70 168.0>, <B91 84.0>, <O111 271.0>, <P25 72.0>, <G11 248.0>, <P90 262.0>, <L65 18.0>, <Q83 144.0>, <G46 258.0>, <L70 123.0>, <C83 134.0>, <J32 102.0>, <E109 478.0>, <L53 152.0>, <F10 10459.0>, <D79 '2003-04'>, <E85 89.0>, <D106 30.0>, <C47 3172.0>, <F84 5345.0>, <J50 1203.0>, <M14 78.0>, <M57 3770.0>, <J94 4518.0>, <B41 'Year'>, <L82 187.0>, <L19 1962.0>, <G12 493.0>, <M79 '2012-13'>, <H111 547.0>, <P24 321.0>, <E10 11011.0>, <O26 158.0>, <F88 59.0>, <O33 254.0>, <J109 678.0>, <G50 1007.0>, <H95 695.0>, <O95 469.0>, <C10 6567.0>, <H61 266.0>, <E101 345.0>, <N96 36.0

In [3]:
observations = observations - tab.excel_ref('B').by_index([38,39,40,41,42,75,76,77,78,79])

In [4]:
Area  = tab.excel_ref('A6').expand(DOWN).is_not_blank() - tab.excel_ref('A').expand(DOWN).by_index([38,39,40,41,42,75,76,77,78,79])
Area 

{<A70 'South Ayrshire'>, <A24 'Moray'>, <A54 'East Renfrewshire'>, <A86 'Dumfries and Galloway'>, <A37 'West Lothian'>, <A108 'South Lanarkshire'>, <A58 'Highland'>, <A22 'Inverclyde'>, <A12 'Dumfries and Galloway'>, <A14 'East Ayrshire'>, <A103 'Perth and Kinross'>, <A111 'West Lothian'>, <A51 'East Ayrshire'>, <A83 'Argyll & Bute'>, <A67 'Renfrewshire'>, <A101 'North Lanarkshire'>, <A74 'West Lothian'>, <A99 'Na h-Eileanan Siar'>, <A52 'East Dunbartonshire'>, <A7 'Aberdeenshire'>, <A20 'Glasgow City'>, <A10 'City of Edinburgh'>, <A28 'Orkney Islands'>, <A62 'Na h-Eileanan Siar'>, <A44 'Aberdeenshire'>, <A26 'North Ayrshire'>, <A90 'East Lothian'>, <A36 'West Dunbartonshire'>, <A81 'Aberdeenshire'>, <A97 'Midlothian'>, <A50 'Dundee City'>, <A92 'Falkirk'>, <A27 'North Lanarkshire'>, <A88 'East Ayrshire'>, <A32 'Shetland Islands'>, <A91 'East Renfrewshire'>, <A56 'Fife'>, <A9 'Argyll & Bute'>, <A46 'Argyll & Bute'>, <A109 'Stirling'>, <A34 'South Lanarkshire'>, <A105 'Scottish Borders'

In [5]:
flow = tab.excel_ref('B').expand(DOWN).by_index([3,40,77])
flow

{<B3 'In-migration - Persons'>, <B40 'In-migration - Males'>, <B77 'In-migration - Females'>}

In [6]:
midyear = tab.excel_ref('B5').expand(RIGHT).is_not_blank()
midyear

{<L5 '2011-12'>, <Q5 '2016-17'>, <N5 '2013-14'>, <O5 '2014-15'>, <C5 '2002-03'>, <F5 '2005-06'>, <D5 '2003-04'>, <E5 '2004-05'>, <B5 '2001-02'>, <G5 '2006-07'>, <H5 '2007-08'>, <K5 '2010-11'>, <P5 '2015-16'>, <M5 '2012-13'>, <I5 '2008-09'>, <J5 '2009-10'>}

In [7]:
Dimensions = [
            HDim(midyear,'Mid Year',DIRECTLY,ABOVE),
            HDim(Area,'Domestic geography', DIRECTLY, LEFT),
            HDim(flow,'flow',CLOSEST,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Flow','Inflow'),
            HDimConst('Age', 'all')
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()
# new_table

In [10]:
new_table = new_table[new_table['Domestic geography'].isnull() == False]

In [11]:
new_table = new_table[new_table['Mid Year'].isnull() == False]

In [12]:
new_table['Mid Year'] = new_table['Mid Year'].map(lambda x: str(x)[0:4]) + '-06-30T00:00:00/P1Y'

In [13]:
new_table['Sex'] = new_table['flow'].map(lambda x: str(x)[15:])

In [14]:
new_table['Value'] = new_table['OBS'].astype(int)

In [15]:
new_table['Sex'] = new_table['Sex'].map(
    lambda x: {
        'Persons' : 'T', 
        'Females' : 'F',
        'Males': 'M' 
        }.get(x, x))



In [16]:
new_table['Foreign geography'] = 'nrs/overseas'

In [17]:
new_table = new_table[['Domestic geography','Foreign geography','Mid Year','Sex','Age','Flow','Measure Type','Value','Unit']]